# Huc 12 Predictors
## Test 19020302 HUC 8 incl chugach nforest
All Huc 12s- Intersect w/catchments and find catchment (find downstream most catchment) with most upstream contribution and build watersheds for all ds catchments

Keep -
mnwtd_slope
% lake
summer precip - Runs in R but need downstream point (centroid of ds catchment)
predict for huc8

Visualize by HUC12 - Join data back to Huc12

## Section
Import modules
Set/Create Workspaces
 * Create Huc12 Catchment dataset to store catchments that will have watersheds generated
 * Crate Catchment point dataset to store catchment centroids
Collect Data
 * Set up data dictionary with VPU as key and store path to source data
    * Slope - Can link to already created slope rasters from original covariates worflow
    * NHDPlus Waterbodies - Can link to waterbody feature clase from original covariate workflow
Merge Data
 * Merge data together and or copy to local/in_memory and check that all projections match



In [1]:
import os, arcpy, sys,datetime
arcpy.env.overwriteOutput = True
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

print('imports complete')
print(f'sys paths {sys.path}')
print (datetime.datetime.now())


imports complete
sys paths ['C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF\\data_preparation\\sensitivity_drivers\\geomorphology', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcPy', 'C:\\Users\\dwmerrigan\\Documents\\GitHub\\AKSSF', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\python37.zip', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\DLLs', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3', '', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin\\Python\\envs\\arcgispro-py3\\lib\\site-packages', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\bin', 'C:\\Users\\dwmerrigan\\AppData\\Local\\Programs\\ArcGIS\\Pro\\Resources\\ArcToolbox\\Scripts', 'C:\\Users\\dwmerrigan\\AppDat

In [2]:
# Function to add key, value pairs to dictionary
def append_value(dict_obj, key, value):
    # Check if key exist in dict or not
    if key in dict_obj:
        # Key exist in dict.
        # Check if type of value of key is list or not
        if not isinstance(dict_obj[key], list):
            # If type is not list then make it list
            dict_obj[key] = [dict_obj[key]]
        # Append the value in list
        dict_obj[key].append(value)
    else:
        # As key is not in dict,
        # so, add key-value pair
        dict_obj[key] = value

## Collect NHDPlus Datasets

In [3]:
#Create Dictionary to relate datasets to vpu
vpuDict = {}
# Walk through folders and subfolders to collect vpus

#Path to folder with NHDPlus data separated by vpu
nhdPlusfol = r'D:\Basedata\NHDPlus'
rois = ['Cook_Inlet','Copper_River']
vpuList = []
regDict = {}

#NHD folders - Update to new data downloaded Nov 8, 2021
for roi in rois:
    arcpy.env.workspace = os.path.join(nhdPlusfol,roi)
    gdbs = arcpy.ListWorkspaces('NHDPLUS*','FileGDB')
    #print(gdbs)
    Cats = []
    VAA = []
    waterbodies = []
    flowlines = []

    for gdb in gdbs:
        arcpy.env.workspace = gdb
        huc = gdb[-20:-12]
        #print(huc)
        vpuList.append(huc)
        append_value(vpuDict,huc,gdb)
        datasets = arcpy.ListDatasets(feature_type='feature')
        for ds in datasets:
            for fc in arcpy.ListFeatureClasses(feature_dataset=ds):
                if fc == "NHDFlowline":
                    flowpath = os.path.join(arcpy.env.workspace, ds, fc)
                    flowname = 'flowline_' + str(huc)
                    append_value(vpuDict,huc,flowpath)
                #     print(f'Copying {flowname}...')
                #     flowcopypath = os.path.join(scratch, flowname)
                #     arcpy.management.CopyFeatures(flowpath, flowcopypath)
                #     flowlines.append(flowcopypath)
                elif fc == "NHDWaterbody":
                    waterbodyname = 'waterbody_' + str(huc)
                    #waterbodycopypath = os.path.join(scratch, waterbodyname)
                    waterpath = os.path.join(arcpy.env.workspace, ds, fc)
                    #print(f'Copying {waterbodyname}...')
                    append_value(vpuDict,huc,waterpath)
                    # arcpy.management.CopyFeatures(waterpath, waterbodycopypath)
                    # waterbodies.append(waterbodycopypath)
                elif fc == "NHDPlusCatchment":
                    catchpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, catchpath)
                elif fc == "WBDHU12":
                    hucpath = os.path.join(arcpy.env.workspace, ds, fc)
                    append_value(vpuDict, huc, hucpath)

        vaapath = os.path.join(arcpy.env.workspace, "NHDPlusFlowlineVAA")
        #print(f'Vaa table at {vaapath} exists = {arcpy.Exists(vaapath)}')
        append_value(vpuDict,huc,vaapath)
        append_value(vpuDict,huc,roi)

vpuDict


{'19020202': ['D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\NHDPlus\\NHDPlusCatchment',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\WBD\\WBDHU12',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\Hydrography\\NHDFlowline',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\Hydrography\\NHDWaterbody',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020202_HU8_GDB.gdb\\NHDPlusFlowlineVAA',
  'Cook_Inlet'],
 '19020301': ['D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\NHDPlus\\NHDPlusCatchment',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\WBD\\WBDHU12',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb\\Hydrography\\NHDFlowline',
  'D:\\Basedata\\NHDPlus\\Cook_Inlet\\NHDPLUS_H_19020301_HU8_GDB.gdb

In [4]:
# Set data dir equal to directory containing the AKSSF regional sub-folders.
akssf_data_dir = r'D:\\GIS\\AKSSF'
nhd_data_dir = r'D:\\Basedata\\NHDPlus'

# Create dictionaries
nhdDict = {}
tauDict = {}
inputDict = {"'":"",'"':""}
scrDict = {}

def replace_all(userinput, dic):
    for i, j in dic.items():
        userinput = userinput.replace(i, j)
    return userinput


# Create scratch workspace
while True:
    try:
        userinput = replace_all((input('Input drive or directory to create scratch workspaces ex. \'W:\\GIS\\\'\n') or 'D:\\GIS\\'),inputDict)
        if not arcpy.Exists(userinput):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            scratchdrive = userinput
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

huc12cv_scratchgdb = os.path.normpath(os.path.join(scratchdrive,r"huc12_Outputs\\huc12_scratch.gdb"))
huc12cv_scratchfol = os.path.dirname(huc12cv_scratchgdb)
print(f'Scratch folder will be created {huc12cv_scratchfol}\nScratch GDB at {huc12cv_scratchgdb} ')

# Specify path to AWC events fc
while True:
    try:
        userinput2 = replace_all((input('Input path to awc events feature class or shapefile. \'"J:\\GIS_data\\biota\\Aquatic\\Fauna\\AWC\\2021_Species_LifeStage.gdb\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs"\'\n') or
                                  'D:\\Basedata\\AWC\\AWC_2021_SpeciesEvents.gdb\\awcEventArcs'), inputDict)
        if not arcpy.Exists(userinput2):
            print('Path specified does not exist!\nPlease re-enter a valid path')
            continue
        else:
            awc_events = userinput2
            break
    except arcpy.ExecuteError:
            print(arcpy.GetMessages())

print(f'Using AWC events from {awc_events}\n {"-"*100}')

# Create Scratch Workspaces and add to dictionary
if not arcpy.Exists(huc12cv_scratchfol):
    os.mkdir(huc12cv_scratchfol)
    print(f'Creating scratch folder {huc12cv_scratchfol}')
else:
    print(f'Scratch folder {huc12cv_scratchfol} already exists')
print('----------')

if not arcpy.Exists(huc12cv_scratchgdb):
    arcpy.CreateFileGDB_management(huc12cv_scratchfol,"huc12_scratch.gdb")
    print(f'Creating scratch gdb {huc12cv_scratchgdb}')
else:
    print(f'Scratch folder {huc12cv_scratchgdb} already exists')
print('----------')


# Create VPU output gdbs
for vpu in vpuList:
    vpu_name = os.path.basename(vpu)
    hucscratchgdbname = 'h12_'+ vpu_name + '.gdb'
    hucscratchgdbpath = os.path.join(huc12cv_scratchfol, hucscratchgdbname)
    append_value(scrDict,vpu_name,hucscratchgdbpath)
    if not arcpy.Exists(hucscratchgdbpath):
        arcpy.CreateFileGDB_management(huc12cv_scratchfol,hucscratchgdbname)
        print(f'Creating scratch gdb for vpu {vpu_name} at {hucscratchgdbpath}')
    else:
        print(f'Scratch gdb for {vpu_name} already created at {hucscratchgdbpath}')

    print('----------')
print(f'All scratch workspaces set')
scrDict



Scratch folder will be created D:\GIS\huc12_Outputs
Scratch GDB at D:\GIS\huc12_Outputs\huc12_scratch.gdb 
Using AWC events from D:\Basedata\AWC\AWC_2021_SpeciesEvents.gdb\awcEventArcs
 ----------------------------------------------------------------------------------------------------
Creating scratch folder D:\GIS\huc12_Outputs
----------
Creating scratch gdb D:\GIS\huc12_Outputs\huc12_scratch.gdb
----------
Creating scratch gdb for vpu 19020202 at D:\GIS\huc12_Outputs\h12_19020202.gdb
----------
Creating scratch gdb for vpu 19020301 at D:\GIS\huc12_Outputs\h12_19020301.gdb
----------
Creating scratch gdb for vpu 19020302 at D:\GIS\huc12_Outputs\h12_19020302.gdb
----------
Creating scratch gdb for vpu 19020401 at D:\GIS\huc12_Outputs\h12_19020401.gdb
----------
Creating scratch gdb for vpu 19020402 at D:\GIS\huc12_Outputs\h12_19020402.gdb
----------
Creating scratch gdb for vpu 19020501 at D:\GIS\huc12_Outputs\h12_19020501.gdb
----------
Creating scratch gdb for vpu 19020502 at D:\GI

{'19020202': 'D:\\GIS\\huc12_Outputs\\h12_19020202.gdb',
 '19020301': 'D:\\GIS\\huc12_Outputs\\h12_19020301.gdb',
 '19020302': 'D:\\GIS\\huc12_Outputs\\h12_19020302.gdb',
 '19020401': 'D:\\GIS\\huc12_Outputs\\h12_19020401.gdb',
 '19020402': 'D:\\GIS\\huc12_Outputs\\h12_19020402.gdb',
 '19020501': 'D:\\GIS\\huc12_Outputs\\h12_19020501.gdb',
 '19020502': 'D:\\GIS\\huc12_Outputs\\h12_19020502.gdb',
 '19020503': 'D:\\GIS\\huc12_Outputs\\h12_19020503.gdb',
 '19020504': 'D:\\GIS\\huc12_Outputs\\h12_19020504.gdb',
 '19020505': 'D:\\GIS\\huc12_Outputs\\h12_19020505.gdb',
 '19020601': 'D:\\GIS\\huc12_Outputs\\h12_19020601.gdb',
 '19020602': 'D:\\GIS\\huc12_Outputs\\h12_19020602.gdb',
 '19020800': 'D:\\GIS\\huc12_Outputs\\h12_19020800.gdb',
 '19020101': 'D:\\GIS\\huc12_Outputs\\h12_19020101.gdb',
 '19020102': 'D:\\GIS\\huc12_Outputs\\h12_19020102.gdb',
 '19020103': 'D:\\GIS\\huc12_Outputs\\h12_19020103.gdb',
 '19020104': 'D:\\GIS\\huc12_Outputs\\h12_19020104.gdb'}

## Section 2
### By Vector Processing Unit
Identify downstream-most catchment for each Huc 12
 * Select by location and select catchment with most us contributing area
    * NHDPlus
        * Use update cursor to join TotalDrainageAreaSqKm from vaa table to catchment
        * Find max value from selection and save as outlet catchment for that HUC12
    * TauDEM
        * DSContArea - Drainage area at the downstream end of the link. Generally this is one grid cell upstream of the downstream end because the drainage area at the downstream end grid cell includes the area of the stream being joined.
 * Generate Centroid point and append to centroid dataset
    * Retain cat_id and Huc12-id
 * Append to HUC12 catchment dataset


In [5]:
proclist = vpuList
for vpu in vpuList:
    if vpu in proclist:
        print(f'Huc {vpu} will be processed')
        # Must match names because some items in dictionary are out of order
        for v in vpuDict[vpu]:
            # Inputs
            if 'NHDPlusCatchment' in v:
                cats = v
            elif 'NHDFlowline' in v:
                streams = v
            elif 'NHDWaterbody'in v:
                lakes = v
            elif 'WBDHU12' in v:
                hucs = v
            elif 'NHDPlusFlowlineVAA' in v:
                vaas = v
            elif ':' not in v:
                region = v
        print(vpu)
        print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')

Huc 19020202 will be processed
19020202
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlusFlowlineVAA
region = Cook_Inlet

Huc 19020301 will be processed
19020301
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020301_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020301_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020301_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020301_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020301_HU8_GDB.gdb\NHDPlusF

In [6]:
# List of HUC12s to be processed
import arcpy, operator, time, datetime

outletDict = {}
outletList = []
outletcats = []
outletcatpts = []

# Uncomment to run all
proclist = vpuList
# Run subset
#proclist = ['19020301']

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

for vpu in vpuList:
    if vpu in proclist:
        try:
            # Start vpu timing function
            vpu_start = time.time()
            print(f'Huc {vpu} will be processed')

            # Must match names because some items in dictionary are out of order
            for v in vpuDict[vpu]:
                # Inputs
                if 'NHDPlusCatchment' in v:
                    cats = v
                elif 'NHDFlowline' in v:
                    streams = v
                elif 'NHDWaterbody'in v:
                    lakes = v
                elif 'WBDHU12' in v:
                    hucs = v
                elif 'NHDPlusFlowlineVAA' in v:
                    vaas = v
                elif ':' not in v:
                    region = v
            print(vpu)
            print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')

            #Output names and paths
            outletcatsname = 'HUC_' + str(vpu) +'_out_cats'
            outcatspath = os.path.join(huc12cv_scratchgdb,outletcatsname)
            outletcatptsname = 'HUC_' + str(vpu) +'_out_catspts'
            outcatptspath = os.path.join(huc12cv_scratchgdb,outletcatptsname)
            print (f'Catchments located {cats}')
            print (f'Vaas located {vaas}')
            print (f'NHD waterbodies located {lakes}')
            print (f'Huc12s located {hucs}')

            # Build Value dictionary to relate NHDPlus id to contributing area
            fields = ['NHDPlusID','TotDASqKm']
            valueDict = {r[0]:(r[1:]) for r in arcpy.da.SearchCursor(vaas, fields)}
            hucselect = arcpy.SelectLayerByLocation_management(hucs,'INTERSECT',awc_events,'','NEW_SELECTION')
            print(('*'*100))
            print(f'{arcpy.GetCount_management(hucselect)} of {arcpy.GetCount_management(hucs)} HUC12s in {vpu} intersect awc events input')
            hucFields = [f for f in arcpy.ListFields(hucselect)]
            # for f in hucFields:
            #     print (f'{f.name} is of {f.type} type')
            with arcpy.da.SearchCursor(hucselect,['HUC12','SHAPE@']) as cur:
                for row in cur:
                    print(f'Processing HUC {row[0]}')
                    inhuc = row[1]
                    cat_layer = arcpy.MakeFeatureLayer_management(cats,'cat_layer')
                    # Select by location using awc and huc 12
                    arcpy.SelectLayerByLocation_management(cat_layer,'HAVE_THEIR_CENTER_IN',inhuc,'','NEW_SELECTION')
                    print(f'Finding outlet for HUC {row[0]} out of {arcpy.GetCount_management(cat_layer)} catchments.\n{("*" * 60)}')
                    catList = [r[0] for r in arcpy.da.SearchCursor(cat_layer, 'NHDPlusID')]
                    intersect = list(set(catList).intersection(valueDict))
                    catDict = {i:(valueDict[i]) for i in intersect}
                    # Find Catchment with max drainage area
                    outcatch = max(catDict.items(), key = operator.itemgetter(1))[0]
                    append_value(outletDict, row[0], int(outcatch))
                    outletList.append(int(outcatch))
                del(row)
            del(cur)

            outlet_cats = arcpy.MakeFeatureLayer_management(cats,'outlet_cats')
            out_expression ='"NHDPlusID" IN ' + str(tuple(outletList))
            #print(out_expression)
            outlet_cats_select = arcpy.SelectLayerByAttribute_management(outlet_cats,'NEW_SELECTION', out_expression)
            print(f'Creating copy of {arcpy.GetCount_management(outlet_cats)} outlet catchments for Huc {vpu} at {outcatspath}')
            print(('*'*100))
            arcpy.FeatureClassToFeatureClass_conversion(outlet_cats_select,huc12cv_scratchgdb,outletcatsname)
            cat_points = arcpy.FeatureToPoint_management(outcatspath, outcatptspath, 'INSIDE')
            outletcats.append(outcatspath)
            outletcatpts.append(outcatptspath)

            # End Vpu time
            vpu_stop = time.time()
            vpu_time = int (vpu_stop - vpu_start)
            print(f'{vpu} Elapsed time: ({datetime.timedelta(seconds=vpu_time)})')
            print(f'{"*"*60}')

        except Exception:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

    else:
        print(('-'*100))
        print(f'Huc {vpu} will not be processed')
        print(('-'*100),'\n')

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')


Huc 19020202 will be processed
19020202
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlusFlowlineVAA
region = Cook_Inlet

Catchments located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
Vaas located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlusFlowlineVAA
NHD waterbodies located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDWaterbody
Huc12s located D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\WBD\WBDHU12
************************************************************************************

## Section 3
### Generate watersheds
Iterate over HUC12 catchment dataset and create watersheds
 * Append output wtd to wtd dataset - Link back to catchment using catID (create catIDcon for unique identifier field)

In [ ]:
# Begin generating watersheds
# List of HUC12s to be processed
import arcpy, time, datetime, os
import pandas as pd
from functools import reduce

# steps 4-9 for loop to create watersheds
arcpy.env.workspace = huc12cv_scratchgdb
arcpy.env.overwriteOutput = True
arcpy.env.qualifiedFieldNames = False
sr = arcpy.SpatialReference(3338)  #'NAD_1983_Alaska_Albers'
arcpy.env.outputCoordinateSystem = sr

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

#proclist = ['19020302']
proclist = vpuList
wtdDict = {}

for vpu in vpuList:
    if vpu in proclist:
        try:
            # Start vpu timing function
            vpu_start = time.time()
            wtdList=[]
            print(f'Huc {vpu} will be processed')
            outgdb = scrDict[vpu]
            # Must match names because some items in dictionary are out of order
            for v in vpuDict[vpu]:
                # Inputs
                if 'NHDPlusCatchment' in v:
                    cats = v
                elif 'NHDFlowline' in v:
                    streams = v
                elif 'NHDWaterbody'in v:
                    lakes = v
                elif 'WBDHU12' in v:
                    hucs = v
                elif 'NHDPlusFlowlineVAA' in v:
                    vaas = v
                elif ':' not in v:
                    region = v
            print(vpu)
            print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')

            catscopy = arcpy.CopyFeatures_management(cats,os.path.join(outgdb,'huc_'+str(vpu)+'_cats'))
            # Get list of index names for cats merge and add index if not already created
            index_names = [i.name for i in arcpy.ListIndexes(catscopy)]

            if 'NHDPlusID_index' not in index_names:
                print (f'Creating index for {catscopy}')
                arcpy.AddIndex_management(catscopy,'NHDPlusID','NHDPlusID_index')
            else:
                print(f'{catscopy} Indexed')

            #watersheds feature dataset for storing fcs
            fdatname = 'Huc_' + str(vpu) + '_Watersheds'
            fdat = os.path.join(outgdb, fdatname)
            if not arcpy.Exists(fdat):
                arcpy.management.CreateFeatureDataset(outgdb, fdatname, sr)
            else:
                print(f'{fdat} exists for {vpu}')

            vaa_df1 = pd.DataFrame(arcpy.da.TableToNumPyArray(vaas, ("NHDPlusID", "FromNode", "ToNode", "StartFlag")))
            stream_df = pd.DataFrame(arcpy.da.TableToNumPyArray(streams, ("NHDPlusID", "FType")))
            dfs = [vaa_df1, stream_df]
            vaa_df = reduce(lambda left,right: pd.merge(left,right,on='NHDPlusID',how="outer"), dfs)
            # remove pipelines
            vaa_df = vaa_df[(vaa_df['FType'] != 428 )]
            vaa_df

            c=1

            for id in outletList:
                # Start iter timing function
                iteration_start = time.time()

                print(f"{c}. Starting watershed for: " + str(id))
                rec = [id]
                up_ids = []
                up_ids.append(rec)
                rec_len = len(rec)
                hws_sum = 0

                while rec_len != hws_sum:
                    fromnode = vaa_df.loc[vaa_df["NHDPlusID"].isin(rec), "FromNode"]
                    rec = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "NHDPlusID"]
                    rec_len = len(rec)
                    rec_hws = vaa_df.loc[vaa_df["ToNode"].isin(fromnode), "StartFlag"]
                    hws_sum = sum(rec_hws)
                    up_ids.append(rec)
                #up_ids is a list with more than numbers, use extend to only keep numeric nhdplusids
                newup_ids = []
                for x in up_ids:
                    newup_ids.extend(x)

                tempLayer = "catsLyr"
                expression = '"NHDPlusID" IN ({0})'.format(', '.join(map(str, newup_ids)) or 'NULL')
                arcpy.MakeFeatureLayer_management(cats, tempLayer, where_clause=expression)
                outdis = "memory/wtd_" + str(round(id))
                outwtd = os.path.join(fdat,'wtd_'+ str(id))

                dis = arcpy.Dissolve_management(tempLayer, outdis)
                watershed = arcpy.EliminatePolygonPart_management(dis, outwtd,"PERCENT", "0 SquareKilometers", 90, "CONTAINED_ONLY")
                wtdList.append(outwtd)
                append_value(wtdDict,vpu,outwtd)
                c=c+1

                # Stop iteration timer
                iteration_stop = time.time()
                iter_time = int (iteration_stop - iteration_start)
                print(f'Elapsed time: ({datetime.timedelta(seconds=iter_time)})')
                print(f'{"*"*60}')

            wtd_merge = arcpy.Merge_management(wtdList, os.path.join(huc12cv_scratchgdb,'AKSSF_HUC_'+str(vpu)+'_wtds_merge'),'','ADD_SOURCE_INFO')
            arcpy.AddField_management(wtd_merge,'cat_ID_con','TEXT')
            arcpy.AddField_management(wtd_merge,'cat_ID','DOUBLE')
            arcpy.AddField_management(wtd_merge,'cat_ID_txt','TEXT')
            arcpy.AddField_management(wtd_merge,'NHDPlusID','DOUBLE')
            with arcpy.da.UpdateCursor(wtd_merge,['MERGE_SRC','NHDPlusID','cat_ID_con','cat_ID','cat_ID_txt']) as cur:
                for row in cur:
                    # Pull nhdplus id from merge source and calculate fields
                    nhdplusid= int(row[0].split('\\')[-1:][0].split('_')[1])
                    row[1] = nhdplusid
                    row[2] = region + '_' + str(nhdplusid)
                    row[3] = nhdplusid
                    row[4] = str(nhdplusid)
                    cur.updateRow(row)
                del(row)
            del(cur)

            # Stop iteration timer
            vpu_stop = time.time()
            vpu_time = int (vpu_stop - vpu_start)
            print(f'{vpu} Elapsed time: ({datetime.timedelta(seconds=vpu_time)})')
            print(f'{"*"*60}')

        except:
            e = sys.exc_info()[1]
            print(e.args[0])
            arcpy.AddError(e.args[0])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')


Huc 19020202 will be processed
19020202
cats = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlus\NHDPlusCatchment
hucs = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\WBD\WBDHU12
streams = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDFlowline
lakes = D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\Hydrography\NHDWaterbody
vaas =  D:\Basedata\NHDPlus\Cook_Inlet\NHDPLUS_H_19020202_HU8_GDB.gdb\NHDPlusFlowlineVAA
region = Cook_Inlet

Creating index for D:\GIS\huc12_Outputs\h12_19020202.gdb\huc_19020202_cats
1. Starting watershed for: 75004200000901
Elapsed time: (0:00:01)
************************************************************
2. Starting watershed for: 75004200001724
Elapsed time: (0:00:01)
************************************************************
3. Starting watershed for: 75004200000064
Elapsed time: (0:00:01)
************************************************************
4. Starting watershed for: 7

## Section 4
###Calculate covariates
Iterate over watersheds and calculate the following:
* Mean wtd slope
* % Lake Cover
* Mean LCLD
* % Glacier Cover

In [ ]:
from arcpy.sa import *

# Path to lcld rasters
lcld_folder = r'D:\\Basedata\\LCLD_rasters_archive'

# Create Dictionary to link input raster/fc data from original covariate workflow
dataDict={'Cook_Inlet':[r"D:\\GIS\\AKSSF\\Cook_Inlet\\slope.tif",r"D:\\GIS\\AKSSF\\Cook_Inlet\\Cook_Inlet.gdb\\glaciers"],
          'Copper_River':[r"D:\\GIS\\AKSSF\\Copper_River\\slope.tif",r"D:\\GIS\\AKSSF\\Copper_River\\Copper_River.gdb\\glaciers"],
          'Prince_William_Sound':[r"D:\\GIS\\AKSSF\\Prince_William_Sound\\slope.tif",r"D:\\GIS\\AKSSF\\Prince_William_Sound\\Prince_William_Sound.gdb\\glaciers"]
          }

# Set output gdb
outgdb = huc12cv_scratchgdb

# Separate data by
nhdplus_dat = ['Cook_Inlet','Copper_River']
tauDem_dat = ['Bristol_Bay', 'Kodiak', 'Prince_William_Sound']

# Lists to store output tables
wtd_lp_tabint_tables = []
wtd_glac_tabint_tables = []
wtd_slope_ztables = []
lcld_Ztables = []

# Start timing function
processStart = time.time()
processStartdt = datetime.datetime.now()

# Iterate over merged watersheds and calculate covariates
arcpy.env.workspace = huc12cv_scratchgdb
for fc in arcpy.ListFeatureClasses('*wtds_merge'):
    vpu = fc[10:18]
    # Must match names because some items in dictionary are out of order
    for v in vpuDict[vpu]:
        # Inputs
        if 'NHDPlusCatchment' in v:
            cats = v
        elif 'NHDFlowline' in v:
            streams = v
        elif 'NHDWaterbody'in v:
            lakes = v
        elif 'WBDHU12' in v:
            hucs = v
        elif 'NHDPlusFlowlineVAA' in v:
            vaas = v
        elif ':' not in v:
            region = v
    print(vpu)
    print(f'cats = {cats}\nhucs = {hucs}\nstreams = {streams}\nlakes = {lakes}\nvaas =  {vaas}\nregion = {region}\n')
    roi = region
    tableid = roi + '_' + str(vpu)
    print(tableid)
    # Set Slope raster from data dictionary
    slope_rast = dataDict[region][0]
    # Set glacier fc from data dictionary
    glac_fc = dataDict[region][1]
    wtd_cur_fields = ['cat_ID_txt', 'cat_ID',"cat_ID_con"]
    if roi in nhdplus_dat:
        lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHDPlus_LakePond_alb"
    # Set data and variables unique to regions with TauDEM Data
    elif roi in tauDem_dat:
        lakes_fc = r"D:\\Basedata\\AKSSF_Basedata\\AKSSF_Basedata.gdb\\AKSSF_NHD_LakesPonds_alb"

    wtds = []

    # Slope variables
    wtd_merge_slope_table_name = tableid + "_Watershed_Merge_SlopeZstats"
    wtd_merge_slope_table_path = os.path.join(outgdb, wtd_merge_slope_table_name)
    # Lakes Ponds variables
    wtd_merge_lp_table_name = tableid + "_Watershed_Merge_LakesPonds"
    wtd_merge_lp_table_path = os.path.join(outgdb, wtd_merge_lp_table_name)
    # Glaciers
    wtd_merge_glac_table_name = tableid + "_Watershed_Merge_Glaciers"
    wtd_merge_glac_table_path = os.path.join(outgdb, wtd_merge_glac_table_name)

    # Watershed slope Zonal Statistics
    print(f'Begin Slope zonal statistics min/mean/max std dev for watersheds and catchments in {roi}'
          f' region')

    # Slope Zonal statistics  for watersheds
    print(f'Calculating {roi} watershed slope zonal stats...')
    arcpy.env.snapRaster = slope_rast
    arcpy.env.cellSize = slope_rast
    wtd_slope_metrics_table = ZonalStatisticsAsTable(in_zone_data = wtd_merge,
                                                    zone_field = wtd_cur_fields[0],
                                                    in_value_raster = slope_rast,
                                                    out_table = wtd_merge_slope_table_path,
                                                    statistics_type='ALL'
                                                    )

    # Add region identifier field for watershed tables                                                )
    arcpy.AddField_management(wtd_slope_metrics_table,'region',field_type='TEXT')
    # Add cat_ID_Con field
    arcpy.AddField_management(wtd_slope_metrics_table,'cat_ID_con',field_type='TEXT')

    # Update region field
    with arcpy.da.UpdateCursor(wtd_slope_metrics_table,['region','cat_ID_txt','cat_ID_con']) as cur:
        for row in cur:
            row[0] = roi
            strval = str(row[1])
            row[2] = roi+"_"+strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)
    # Append watershed slope table to list
    wtd_slope_ztables.append(wtd_slope_metrics_table)

    # Percent Lakes Ponds using Tabulate Intersection for watersheds
    print(f'Begin tabulate intersection between {lakes_fc} and watersheds in {roi} region')
    print('----------')
    wtd_lp_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                        zone_fields=wtd_cur_fields[0],
                                                        in_class_features=lakes_fc,
                                                        out_table=wtd_merge_lp_table_path,
                                                        class_fields='Ftype',
                                                        out_units="SQUARE_METERS"
                                                        )
    # Add region and cat id fields
    arcpy.AlterField_management(wtd_lp_tabint,'PERCENTAGE','wtd_lake_per','wtd_lake_per')
    arcpy.AlterField_management(wtd_lp_tabint,'AREA','wtd_lake_area_sqm','wtd_lake_area_sqm')
    arcpy.AddField_management(wtd_lp_tabint, 'region', field_type='TEXT')
    arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[1], field_type='TEXT')
    arcpy.AddField_management(wtd_lp_tabint, wtd_cur_fields[2], field_type='TEXT')
    wtdlpfields = [f.name for f in arcpy.ListFields(wtd_lp_tabint)]
    print (wtdlpfields)
    with arcpy.da.UpdateCursor(wtd_lp_tabint, wtdlpfields) as cur:
        for row in cur:
            strval = str(row[1])
            row[5] = roi
            row[6] = strval.replace('.0','')
            row[7] = roi +'_'+ strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)

    # Append watershed lakes ponds table to list
    wtd_lp_tabint_tables.append(wtd_lp_tabint)

    # Percent glaciers using Tabulate Intersection for watersheds
    print(f'Begin tabulate intersection between {glac_fc} and watersheds in {roi} region')
    print('----------')
    wtd_glac_tabint = arcpy.TabulateIntersection_analysis(wtd_merge,
                                                        zone_fields=wtd_cur_fields[0],
                                                        in_class_features=glac_fc,
                                                        out_table=wtd_merge_glac_table_path,
                                                        class_fields='O1Region',
                                                        out_units="SQUARE_METERS"
                                                        )
    # Add region and cat id fields
    arcpy.AlterField_management(wtd_glac_tabint,'PERCENTAGE','wtd_glacier_per','wtd_glacier_per')
    arcpy.AlterField_management(wtd_glac_tabint,'AREA','wtd_glacier_area_sqm','wtd_glacier_area_sqm')
    arcpy.AddField_management(wtd_glac_tabint, 'region', field_type='TEXT')
    arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[1], field_type='TEXT')
    arcpy.AddField_management(wtd_glac_tabint, wtd_cur_fields[2], field_type='TEXT')
    wtdglacfields = [f.name for f in arcpy.ListFields(wtd_glac_tabint)]
    print (wtdglacfields)
    with arcpy.da.UpdateCursor(wtd_glac_tabint, wtdglacfields) as cur:
        for row in cur:
            strval = str(row[1])
            row[5] = roi
            row[6] = strval.replace('.0','')
            row[7] = roi +'_'+ strval.replace(".0","")
            # Update
            cur.updateRow(row)
        del(row)
    del(cur)
    # Append watershed percent north table to list
    wtd_glac_tabint_tables.append(wtd_glac_tabint)

    # Begin LCLD calculations
    walk = arcpy.da.Walk(lcld_folder, datatype='RasterDataset')
    for dirpath, dirnames, filenames in walk:
        for filename in filenames:
            raspath = os.path.join(dirpath, filename)
            year = filename[0:4]
            lcld_outname = 'lcld_'+str(year)+'_zStats'
            lcld_outpath = os.path.join(outgdb, lcld_outname)
            print(f'Year: {year} - raster path {raspath}')
            colname = 'wtd_lcld_mn_' + str(year)
            # lcld zonal statistics as table for all akssf watersheds
            print(f'Calculating {filename} zonal stats for all AKSSF watersheds...')
            #arcpy.env.snapRaster = raspath
            #arcpy.env.cellSize = raspath
            try:
                # Begin Zonal Stats
                zstat_start = time.time()
                print(f'Begin zonal stats for {filename}')
                lcld_table = ZonalStatisticsAsTable(in_zone_data = fc,
                                                                zone_field = 'cat_ID_con',
                                                                in_value_raster = raspath,
                                                                out_table = lcld_outpath,
                                                                statistics_type='MEAN'
                                                                )
                # Append zTable to table list
                lcld_Ztables.append(lcld_outpath)
                arcpy.AlterField_management(lcld_table,'MEAN', colname,colname)
                proc_list = [row[0] for row in arcpy.da.SearchCursor(lcld_table,'cat_ID_con')]
                zstat_stop = time.time()
                zstat_time = int (zstat_stop - zstat_start)
                print(f'Zonal Stats for {filename} Elapsed time: ({datetime.timedelta(seconds=zstat_time)})')


            except:
                e = sys.exc_info()[1]
                print(e.args[0])
                arcpy.AddError(e.args[0])

# End timing
processEnd = time.time()
processElapsed = int(processEnd - processStart)
processSuccess_time = datetime.datetime.now()

# Report success
print(f'Process completed at {processSuccess_time.strftime("%Y-%m-%d %H:%M")} '
      f'(Elapsed time: {datetime.timedelta(seconds=processElapsed)})')
print('----------')

## Examine tables and merge/export

In [ ]:
dfs = []
for table in lcld_Ztables:
    tblname = table[-16:]
    print(tblname)
    dfname = tblname + '_arr'
    # Make df
    dfname = pd.DataFrame()
    lcld_field_list = []
    for field in arcpy.ListFields(table):
        lcld_field_list.append(field.name)
        #print(f'{field.name}')
    lcld_arr = arcpy.da.TableToNumPyArray(table, lcld_field_list)
    dfname = pd.DataFrame(lcld_arr)
    dfname = dfname.drop(['OBJECTID','ZONE_CODE', 'AREA', 'COUNT'],axis=1)
    dfname = dfname.set_index('cat_ID_con')
    dfs.append(dfname)

# Merge all data frames together
import numpy as np
from functools import reduce
lcld_df = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
lcld_df

In [ ]:
# Export merged dataframe to csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
lcld_csv_out = os.path.join(outdir,'AKSSF_all_nhd_huc12_wtd_lcld_mn.csv')
lcld_df.to_csv(lcld_csv_out, encoding = 'utf-8')
print('Export dataframe to csv complete')

In [ ]:
# Table names/paths
wtd_per_glac_table_out = os.path.join(outgdb, 'AKSSF_all_nhd_huc12_wtd_glacier_per')
wtd_per_lp_table_out = os.path.join(outgdb, 'AKSSF_all_nhd_huc12_wtd_lakepond_per')
wtd_slope_table_out = os.path.join(outgdb, 'AKSSF_all_nhd_huc12_wtd_slope')
wtd_lcld_table_out = os.path.join(outgdb, 'AKSSF_all_nhd_huc12_wtd_lcld')


# Merge all regional tables together
outtables = []
wtd_slope = arcpy.Merge_management(wtd_slope_ztables, wtd_slope_table_out)
outtables.append(wtd_slope)
wtd_glac = arcpy.Merge_management(wtd_glac_tabint_tables, wtd_per_glac_table_out)
outtables.append(wtd_glac)
wtd_lp = arcpy.Merge_management(wtd_lp_tabint_tables, wtd_per_lp_table_out)
outtables.append(wtd_lp)
print ('Tables merged')
print('----------')

In [ ]:
slopeDict = { 'ZONE_CODE': ('cat_slope_ZONE_CODE', 'wtd_slope_ZONE_CODE'),
         'COUNT': ('cat_slope_COUNT', 'wtd_slope_COUNT'),
          'AREA': ('cat_slope_AREA', 'wtd_slope_AREA'),
          'MIN': ('cat_slope_MIN', 'wtd_slope_MIN'),
          'MAX': ('cat_slope_MAX', 'wtd_slope_MAX'),
          'RANGE': ('cat_slope_RANGE', 'wtd_slope_RANGE'),
          'MEAN': ('cat_slope_MEAN', 'wtd_slope_MEAN'),
          'STD': ('cat_slope_STD', 'wtd_slope_STD'),
          'SUM': ('cat_slope_SUM', 'wtd_slope_SUM'),
          'VARIETY': ('cat_slope_VARIETY', 'wtd_slope_VARIETY'),
          'MAJORITY': ('cat_slope_MAJORITY', 'wtd_slope_MAJORITY'),
          'MINORITY': ('cat_slope_MINORITY', 'wtd_slope_MINORITY'),
          'MEDIAN': ('cat_slope_MEDIAN', 'wtd_slope_MEDIAN'),
          'PCT90': ('cat_slope_PCT90', 'wtd_slope_PCT90')
         }
# Rename fields for slope tables
for field in arcpy.ListFields(wtd_slope):
    keyval = field.name
    if keyval in slopeDict:
        newname = slopeDict[keyval][1]
        newalias = slopeDict[keyval][1]
        print (keyval, newname)
        arcpy.AlterField_management(wtd_slope, keyval, newname, newalias)


In [ ]:
# Export copies of dbf tables as csv
outdir = r"C:\Users\dwmerrigan\Documents\GitHub\AKSSF\data_preparation\sensitivity_drivers"
for table in outtables:
    tablename = arcpy.Describe(table).basename + ".csv"
    tablepath = os.path.join(outdir,tablename)
    print( tablepath)
    arcpy.conversion.TableToTable(table, outdir, tablename)

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format # only display 2 decimal places
# list to store covariate data frames
dfs = []

In [ ]:
# Make watershed slope df
wtd_sl_df = pd.DataFrame()
wtd_sl_field_list = []
for field in arcpy.ListFields(wtd_slope):
    wtd_sl_field_list.append(field.name)
wtd_sl_arr = arcpy.da.TableToNumPyArray(wtd_slope, wtd_sl_field_list)
wtd_sl_df = pd.DataFrame(wtd_sl_arr)
wtd_sl_df = wtd_sl_df.drop(["OBJECTID", "wtd_slope_ZONE_CODE"],axis=1)
wtd_sl_df = wtd_sl_df.set_index('cat_ID_con')
dfs.append(wtd_sl_df)
wtd_sl_df

In [ ]:
# Make watershed lakes df
wtd_lp_df = pd.DataFrame()
wtd_lp_field_list = []
for field in arcpy.ListFields(wtd_lp):
    wtd_lp_field_list.append(field.name)
wtd_lp_arr = arcpy.da.TableToNumPyArray(wtd_lp, wtd_lp_field_list)
wtd_lp_df = pd.DataFrame(wtd_lp_arr)
wtd_lp_df = wtd_lp_df.drop("OBJECTID",axis=1)
wtd_lp_df = wtd_lp_df.set_index('cat_ID_con')
dfs.append(wtd_lp_df)
wtd_lp_df

In [ ]:
# Make watershed glacier df
wtd_glac_df = pd.DataFrame()
wtd_glac_field_list = []
for field in arcpy.ListFields(wtd_glac):
    wtd_glac_field_list.append(field.name)
wtd_glac_arr = arcpy.da.TableToNumPyArray(wtd_glac, wtd_glac_field_list)
wtd_glac_df = pd.DataFrame(wtd_glac_arr)
wtd_glac_df = wtd_glac_df.drop("OBJECTID",axis=1)
wtd_glac_df = wtd_glac_df.set_index('cat_ID_con')
dfs.append(wtd_glac_df)
wtd_glac_df

In [ ]:
# Merge all data frames together
import numpy as np
from functools import reduce
df_final = reduce(lambda left,right: pd.merge(left,right,on='cat_ID_con',how="outer"), dfs)
#Generate unique column names
def uniquify(df_final):
    seen = set()
    for item in df_final:
        fudge = 1
        newitem = item
        while newitem in seen:
            fudge += 1
            newitem = "{}_{}".format(item, fudge)
        yield newitem
        seen.add(newitem)
df_final.columns = list(uniquify(df_final))

#List of final columns in the order to output
final_cols = ['cat_ID_txt', 'wtd_slope_COUNT', 'wtd_slope_AREA', 'wtd_slope_MIN', 'wtd_slope_MAX','wtd_slope_RANGE', 'wtd_slope_MEAN', 'wtd_slope_STD', 'wtd_slope_SUM', 'wtd_slope_MEDIAN', 'wtd_slope_PCT90',
              'wtd_lake_area_sqm', 'wtd_lake_per', 'wtd_glacier_area_sqm', 'wtd_glacier_per' ]
#Create list of duplicate column names and drop
drop_cols = ['cat_ID_txt_y', 'region_y','FType', 'O1Region','cat_ID_y','region','cat_ID_x','cat_ID_txt']
df_final.drop(columns=drop_cols, axis = 1, inplace=True)

#rename columns
df_final.rename({'cat_ID_txt_x':'cat_ID_txt','region_x':'region'},axis=1, inplace=True)
#Recalculate cat_ID
df_final['cat_ID'] = df_final['cat_ID_txt'].astype(np.float64)
df_final.index.is_unique
# reorder cols
df_final = df_final.reindex(columns=final_cols)
df_final

In [ ]:
# Export merged dataframe to csv
huccov_csv_out = os.path.join(outdir,'AKSSF_ALL_NHD_HUC12_Covariates.csv')
df_final.to_csv(huccov_csv_out, encoding = 'utf-8')
print('Export all covariates dataframe to csv complete')
